In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from google.colab import drive

In [2]:
# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define dataset paths
base_path = "/content/drive/My Drive/Dataset"
folders = ["Armyworms", "Cabbage Loopers"]
folder_paths = [os.path.join(base_path, folder) for folder in folders]

# Step 3: Initialize images and labels lists
images = []
labels = []

Mounted at /content/drive


In [3]:
# Step 4: Function to load images
def load_images_from_folder(folder_path, label, images_list, labels_list):
    for filename in tqdm(os.listdir(folder_path), desc=f"Loading {os.path.basename(folder_path)}"):
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path)

        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            img = cv2.resize(img, (224, 224))  # Resize to (224, 224)
            images_list.append(img)
            labels_list.append(label)

# Step 5: Load images with labels
for label, folder in enumerate(folder_paths):
    load_images_from_folder(folder, label, images, labels)

# Step 6: Convert lists to NumPy arrays and normalize
images = np.array(images, dtype=np.float32) / 255.0  # Normalize pixel values
labels = np.array(labels)
print(labels)

Loading Cabbage Loopers: 100%|██████████| 104/104 [00:18<00:00,  5.68it/s]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [4]:
# Step 7: Shuffle
images, labels = shuffle(images, labels, random_state=42)

# Step 8: Confirm labels
print("Labels:", labels)

# Step 9: Split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)



Labels: [0 0 1 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 0 1
 1 0 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 1 1 0 1
 1 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1 0 0 1 1 0 1 1 0
 1 0 0 1 0 0 1 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0
 0 1 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 1 1 1
 1 1 0 0 1 1 0 1 0 1 0 0 1 1]


In [5]:
# Verify the shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotate images up to 20 degrees
    width_shift_range=0.1,  # Shift width by 10%
    height_shift_range=0.1,  # Shift height by 10%
    shear_range=0.1,  # Shear transformation
    zoom_range=0.1,  # Zoom up to 10%
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill missing pixels with nearest value
)

X_train shape: (159, 224, 224, 3), y_train shape: (159,)
X_test shape: (40, 224, 224, 3), y_test shape: (40,)


In [6]:
# Step 10: Build Custom CNN Model
model = Sequential([
    # First Conv Block
    Conv2D(128, (5, 5), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Second Conv Block
    Conv2D(64, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Third Conv Block
    Conv2D(32, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten and Fully Connected Layers
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),  # Reduces overfitting
    Dense(1, activation='sigmoid')  # Output Layer
])

# Step 11: Compile Model
learning_rate = 0.004  # Custom learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Step 12: Train Model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 13: Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Step 14: Save Model
model.save("/content/drive/My Drive/Dataset/new_cnn_model.h5")
print("Custom CNN Model saved successfully!")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 16s/step - accuracy: 0.5018 - loss: 3.3124 - val_accuracy: 0.4750 - val_loss: 1.5468
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 78s 16s/step - accuracy: 0.5478 - loss: 1.0239 - val_accuracy: 0.5250 - val_loss: 0.7447
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 81s 16s/step - accuracy: 0.5273 - loss: 0.7207 - val_accuracy: 0.4750 - val_loss: 0.6933
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 90s 17s/step - accuracy: 0.5141 - loss: 0.6928 - val_accuracy: 0.4750 - val_loss: 0.6936
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 146s 17s/step - accuracy: 0.5515 - loss: 0.6924 - val_accuracy: 0.4750 - val_loss: 0.6943
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 137s 17s/step - accuracy: 0.5063 - loss: 0.6931 - val_accuracy: 0.4750 - val_loss: 0.6948
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 136s 15s/step - accuracy: 0.5580 - loss: 0.6910 - val_accuracy: 0.4750 - val_loss: 0.6956
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 83s 16s/step - accuracy: 0.5588 - loss: 0.6896 - val_accuracy: 0.4750 - val_loss: 0.69

Test Accuracy: 0.47
Custom CNN Model saved successfully!
